<a href="https://colab.research.google.com/github/yanann11/nebius_llm_course/blob/main/topic1/1.5_how_to_choose_an_llm_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Engineering Essentials by Nebius Academy

Course github: [link](https://github.com/Nebius-Academy/LLM-Engineering-Essentials/tree/main)

The course is in development now, with more materials coming soon.

# 1.5. How to choose an LLM

# Practice solutions

## Task 1. Advanced MMLU testing

In this task, you'll need to upgrade the `MMLUEvaluator` class to also compare:

1. **Average latency** (that is, average time to solve a problem). Add `'avg_inference_time'` to the outputs of `run_evaluation`. Make sure that you only measure the timing of producing the competion, not of the whole `evaluate_single_question` running - this will be especially relevant when we add the translation phase.

  In theory, average latency would reflect the LLM's size and average answer length. Note that for rarer languages tokens will be smaller, and as consequence the answer length in tokens will be larger (even if visible answer length will be comparable with English). This will, of course, contribute to the latency.
  
  In reality though, average latency also highly depends on the *API provider* or your own deployment efforts. APIs may have periods of higher or lower latency; they also introduce optimizations which might work or not work, depending on the architectural details of different LLMs.

2. **Multilingual proficiency**. Almost every Q&A-related benchmark exposes LLMs to questions in English, because

  (a) gathering data in English is much easier than in any other language,

  (b) English benchmarks are relevant to larger portion of the AI community,

  (c) the numbers look better when you check things in English :)

  But in this task you'll try to add a `language` parameter to the `run_evaluation` mehtod. When it's `None`, the LLM will be tested on the original English questions and answers; otherwise, the specified language will be used. If you have time, try several MMLU topics and several languages. How much will the quality fall in comparison with English?

  You'll need to use an LLM for translation of questions and answers. Some guidelines you might have in mind:

  * Choose the translator LLM wisely. We suggest using a powerful one, because otherwise you'll see the effects of translation, not of the language choice. If you have access to OpenAI or Anthropic API, leveraging their models won't hurt. If you use long-reasoning models such as `o4`, `DeepSeek R1`, or `Qwen3`, don't forget to increase the `max_tokens` parameter for the translator call, because these models tend to be wordy.
  * Assess the translation quality before you start running your benchmarks. For that, choose the language you or your friends know well.
  * You might want to cache the translations if you're going to test multiple LLMs.
  * Generally, there are two strategies of translation. You can either feed the whole

    ```
    QUESTION: {question}

    ANSWER OPTIONS:
    A: {A}
    B: {B}
    C: {C}
    D: {D}
    ```
  
    structure to the translator or translate the question and the answer options separately. The second option will be slightly more expensive. The first one might be tricky, because you'll need the LLM to strictly obey the format and abstain from commenting on the answers or a potential solution. It can be achieved through clever prompting, but the better strategy is using either few-shot examples or structured generation which will be the discussed in Topic 2. So, for now, we suggest separate translation.

**Solution**. Here's out implementation. It translates questions and answers separately, while providing an option of not translating answers (which is perfectly reasonable for math questions). It also caches translations. For demonstrational purposes we use **Llama-3.1-405B** as a translator model.

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [ ]:
import pandas as pd
from typing import List, Dict, Tuple, Optional
import json
from pathlib import Path
import numpy as np
from tqdm import tqdm
import time
import os
import hashlib
import pickle
from openai import OpenAI

from datasets import load_dataset

nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

class MMLUEvaluator:
    def __init__(self, system_prompt: str = None, prompt: str = None,
                 topic: str = "high_school_mathematics",
                 translator_client=nebius_client, translator_model=None,
                 cache_dir: str = "translation_cache"):
        """
        Initialize the MMLU evaluator.

        Args:
            system_prompt: Optional system prompt for the model
            prompt: Custom prompt for the model
            topic: Which topic to choose
            translator_client: Client for translation model
            translator_model: Model to use for translation
            cache_dir: Directory to store translation cache
        """

        self.topic = topic
        self.topic_prettified = topic.replace("_", " ")
        self.system_prompt = system_prompt or f"You are an expert in {self.topic_prettified}."
        self.translator_client = translator_client
        self.translator_model = translator_model

        # Setup cache directory and translation cache
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True, parents=True)
        self.translation_cache = self._load_translation_cache()

        # Simple translation prompt for a single text
        self.simple_translation_prompt = """
Translate the following text from English to {language}.
Return only the translated content enclosed within <translation> and </translation> tags.
Leave formulas and mathematical notations as they are.

Text to translate: {text}
"""

        self.prompt = """You are given a question in {topic_prettified} with four answer options labeled by A, B, C, and D.
You need to ponder the question and justify the choice of one of the options A, B, C, or D.
At the end, do write the chosen answer option A, B, C, D after #ANSWER:
Now, take a deep breath and work out this problem step by step. If you do well, I'll tip you 200$.

QUESTION: {question}

ANSWER OPTIONS:
A: {A}
B: {B}
C: {C}
D: {D}
"""

        self.questions, self.choices, self.answers = self.load_mmlu_data(topic=self.topic)

    def _get_cache_key(self, text: str, language: str, model: Optional[str] = None) -> str:
        """
        Generate a unique cache key for a text and language combination.

        Args:
            text: Text to translate
            language: Target language
            model: Optional model identifier

        Returns:
            A unique hash key for the translation
        """
        model_str = model or self.translator_model or "default_model"
        content = f"{text}_{language}_{model_str}"
        return hashlib.md5(content.encode()).hexdigest()

    def _get_cache_path(self) -> Path:
        """Get the path to the translation cache file."""
        topic_safe = self.topic.replace("/", "_")
        return self.cache_dir / f"{topic_safe}_translation_cache.pkl"

    def _load_translation_cache(self) -> Dict:
        """Load the translation cache from disk if it exists."""
        cache_path = self._get_cache_path()
        if cache_path.exists():
            try:
                with open(cache_path, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"Error loading translation cache: {e}")
                return {}
        return {}

    def _save_translation_cache(self):
        """Save the translation cache to disk."""
        cache_path = self._get_cache_path()
        try:
            with open(cache_path, 'wb') as f:
                pickle.dump(self.translation_cache, f)
        except Exception as e:
            print(f"Error saving translation cache: {e}")

    def load_mmlu_data(self, topic: str) -> pd.DataFrame:
        """
        Load MMLU test data on a given topic.

        Args:
            topic: Which topic to choose

        Returns:
            DataFrame with questions and answers
        """

        dataset = load_dataset("cais/mmlu", topic, split="test")

        dataset = dataset
        dataset = pd.DataFrame(dataset)

        # Load questions and choices separately
        questions = dataset["question"]
        choices = pd.DataFrame(
            data=dataset["choices"].tolist(), columns=["A", "B", "C", "D"]
        )
        # In the dataset, true answer labels are in 0-3 format;
        # We convert it to A-D
        answers = dataset["answer"].map(lambda ans: {0: "A", 1: "B", 2: "C", 3: "D"}[ans])

        return questions, choices, answers

    def translate_text(self, text: str, language: str) -> Tuple[str, str]:
        """
        Translate a single piece of text to the target language.
        Uses cache if available, otherwise calls the translation model.

        Args:
            text: The text to translate
            language: Target language for translation

        Returns:
            Tuple of (translated_text, raw_response) or (original_text, error_message) if translation fails
        """
        if not language or not self.translator_client or not self.translator_model:
            return text, "No translation requested"

        # Generate cache key
        cache_key = self._get_cache_key(text, language, self.translator_model)

        # Check if we have this translation in cache
        if cache_key in self.translation_cache:
            cached_result = self.translation_cache[cache_key]
            print(f"Using cached translation for: {text[:30]}...")
            return cached_result["translation"], cached_result["raw_response"]

        # Not in cache, perform translation
        try:
            translation_prompt = self.simple_translation_prompt.format(
                language=language,
                text=text
            )

            translation_response = answer_with_llm(
                prompt=translation_prompt,
                system_prompt=f"You are a professional translator from English to {language}.",
                client=self.translator_client,
                model=self.translator_model,
                prettify=False
            )

            # Extract translation from between tags
            try:
                translation = translation_response.split('<translation>')[1].split('</translation>')[0].strip()

                # Save to cache
                self.translation_cache[cache_key] = {
                    "translation": translation,
                    "raw_response": translation_response,
                    "timestamp": time.time()
                }

                # Persist cache to disk
                self._save_translation_cache()

                return translation, translation_response
            except:
                error_msg = f"Failed to extract translation for: {text[:30]}..."
                print(error_msg)
                return text, f"{error_msg}\nRaw response: {translation_response}"

        except Exception as e:
            error_msg = f"Translation error: {e}"
            print(error_msg)
            return text, error_msg

    def translate_problem(self, question: str, choices: Dict[str, str],
                          language: str, translate_answers: bool = True) -> Tuple[str, Dict[str, str], Dict]:
        """
        Translate the problem to the target language.

        Args:
            question: The question to translate
            choices: The answer choices to translate
            language: Target language for translation
            translate_answers: Whether to translate answer options

        Returns:
            Tuple of (translated_question, translated_choices, translation_logs)
        """
        translation_logs = {
            "question": {
                "original": question,
                "translated": None,
                "raw_response": None
            },
            "choices": {}
        }

        if not language or not self.translator_client or not self.translator_model:
            return question, choices, translation_logs

        # Translate the question
        translated_question, raw_response = self.translate_text(question, language)
        translation_logs["question"]["translated"] = translated_question
        translation_logs["question"]["raw_response"] = raw_response

        # If we're not translating answers, return just the translated question
        if not translate_answers:
            for key, value in choices.items():
                translation_logs["choices"][key] = {
                    "original": value,
                    "translated": value,  # Not translated
                    "raw_response": "Answer translation disabled"
                }
            return translated_question, choices, translation_logs

        # Translate each answer option individually
        translated_choices = {}
        for key, value in choices.items():
            translated_choice, raw_response = self.translate_text(value, language)
            translated_choices[key] = translated_choice

            # Log the translation
            translation_logs["choices"][key] = {
                "original": value,
                "translated": translated_choice,
                "raw_response": raw_response
            }

        return translated_question, translated_choices, translation_logs

    def extract_answer(self, solution: str) -> str:
        """
        Extract the letter answer from model's response.

        Args:
            response: Raw model response

        Returns:
            Extracted answer letter (A, B, C, D, or Failed to parse)
        """
        # Look for a single letter answer in the response
        try:
            answer = solution.split('#ANSWER:')[1].strip()
        except:
            answer = "Failed to parse"
        return answer

    def evaluate_single_question(self, question: str, choices: Dict[str, str],
                                 correct_answer: str,
                                 client, model, language=None,
                                 translate_answers=True) -> Tuple[bool, str, str, float, Dict]:
        """
        Evaluate a single question.

        Args:
            question: Formatted question string
            correct_answer: Correct answer letter
            language: Target language for translation (None for English)
            translate_answers: Whether to translate answer options (default: True)

        Returns:
            Tuple of (is_correct, extracted_answer, model_response, inference_time, translation_logs)
        """
        translation_logs = None

        try:
            # Translate if needed
            if language:
                translated_question, translated_choices, translation_logs = self.translate_problem(
                    question, choices, language, translate_answers
                )
                # Use translated content
                question = translated_question
                choices = translated_choices

            formatted_prompt = self.prompt.format(
                topic_prettified=self.topic_prettified,
                question=question,
                A=choices['A'], B=choices['B'], C=choices['C'], D=choices['D']
            )

            # Measure inference time
            start_time = time.time()
            model_response = answer_with_llm(
                prompt=formatted_prompt,
                system_prompt=self.system_prompt,
                client=client,
                model=model,
                prettify=False
            )
            end_time = time.time()
            inference_time = end_time - start_time

            answer = self.extract_answer(model_response)
            is_correct = (answer.upper() == correct_answer.upper())
            return is_correct, answer, model_response, inference_time, translation_logs
        except Exception as e:
            print(f"Error evaluating question: {e}")
            return False, None, None, 0, translation_logs

    def run_evaluation(self, client=nebius_client, model=None,
                       n_questions=50, language=None,
                       translate_answers=True) -> Dict:
        """
        Run evaluation of a given model on the first n_questions.

        Args:
            client: Which client to use (OpenAI or Nebius)
            model: Which model to use
            n_questions: How many first questions to take
            language: Target language for translation (None for English)
            translate_answers: Whether to translate answer options (default: True)

        Returns:
            Dictionary with evaluation metrics
        """
        evaluation_log = []
        correct_count = 0
        total_time = 0
        translation_logs = []

        if n_questions:
            n_questions = min(n_questions, len(self.questions))
        else:
            n_questions = len(self.questions)

        for i in tqdm(range(n_questions)):
            is_correct, answer, model_response, inference_time, trans_log = self.evaluate_single_question(
                question=self.questions[i],
                choices=self.choices.iloc[i],
                correct_answer=self.answers[i],
                client=client,
                model=model,
                language=language,
                translate_answers=translate_answers
            )

            if is_correct:
                correct_count += 1

            total_time += inference_time

            log_entry = {
                'question_id': i,
                'original_question': self.questions[i],
                'original_choices': self.choices.iloc[i].to_dict(),
                'correct_answer': self.answers[i],
                'model_answer': answer,
                'model_response': model_response,
                'is_correct': is_correct,
                'inference_time': inference_time,
                'translation_log': trans_log
            }

            evaluation_log.append(log_entry)

            # Add to translation logs if available
            if trans_log:
                translation_logs.append({
                    'question_id': i,
                    'translation_log': trans_log
                })

        accuracy = correct_count / n_questions
        avg_inference_time = total_time / n_questions if n_questions > 0 else 0

        evaluation_results = {
            'accuracy': accuracy,
            'avg_inference_time': avg_inference_time,
            'total_inference_time': total_time,
            'evaluation_log': evaluation_log,
            'translation_logs': translation_logs,
            'language': language,
            'translate_answers': translate_answers,
            'cache_stats': {
                'cache_size': len(self.translation_cache),
                'cache_path': str(self._get_cache_path())
            }
        }

        return evaluation_results

    def clear_translation_cache(self):
        """Clear the translation cache and delete the cache file."""
        self.translation_cache = {}
        cache_path = self._get_cache_path()
        if cache_path.exists():
            cache_path.unlink()
            print(f"Deleted translation cache file: {cache_path}")

In [ ]:
evaluator = MMLUEvaluator(
    topic="high_school_mathematics", translator_model="meta-llama/Meta-Llama-3.1-405B-Instruct"
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/6.99k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/4.50k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/270 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Now, we can run evaluation for several models. The function `evaluator.run_evaluation` will return both classification accuracy and the full log containing the model's responses and the extracted answers.

In [ ]:
results = evaluator.run_evaluation(model="meta-llama/Meta-Llama-3.1-70B-Instruct",
                         n_questions=50, language=None)
print(f'\nAccuracy: {results["accuracy"]}')
print(f'\nAverage inference time: {results["avg_inference_time"]:.1f} sec')

100%|██████████| 50/50 [07:43<00:00,  9.27s/it]


Accuracy: 0.76

Average inference time: 9.3 sec


In [ ]:
results = evaluator.run_evaluation(model="meta-llama/Meta-Llama-3.1-70B-Instruct",
                         n_questions=50, language="French", translate_answers=False)
print(f'\nAccuracy: {results["accuracy"]}')
print(f'\nAverage inference time: {results["avg_inference_time"]:.1f} sec')

 76%|███████▌  | 38/50 [27:04<05:50, 29.23s/it]

Using cached translation for: What is the smallest positive ...


 96%|█████████▌| 48/50 [31:47<00:51, 25.73s/it]

Using cached translation for: John divided his souvenir hat ...


100%|██████████| 50/50 [32:14<00:00, 38.69s/it]


Accuracy: 0.34

Average inference time: 36.7 sec


In [ ]:
results["evaluation_log"][5]["translation_log"]

{'question': {'original': 'John divided his souvenir hat pins into two piles. The two piles had an equal number of pins. He gave his brother one-half of one-third of one pile. John had 66 pins left. How many pins did John originally have?',
  'translated': "John a divisé ses épingles de chapeau souvenir en deux tas. Les deux tas avaient un nombre égal d'épingles. Il a donné à son frère la moitié du tiers d'un tas. John avait 66 épingles en reste. Combien d'épingles John avait-il à l'origine ?",
  'raw_response': "<translation>John a divisé ses épingles de chapeau souvenir en deux tas. Les deux tas avaient un nombre égal d'épingles. Il a donné à son frère la moitié du tiers d'un tas. John avait 66 épingles en reste. Combien d'épingles John avait-il à l'origine ?</translation>"},
 'choices': {'A': {'original': '396',
   'translated': '396',
   'raw_response': 'Answer translation disabled'},
  'B': {'original': '72',
   'translated': '72',
   'raw_response': 'Answer translation disabled'}